# Functions and State


So far our programs are side-effect free. That means all program that terminate, any sequence of actions would give the same result. This was reflected also on the substitution model of the computation.
We are going to broaden our notion of functions, to work with mutable state.

In substitution model, the programs can be evaluated by rewriting the program text
The most important rewrite rule covers function applications:

```scala
def f(x1, ..., xn) = B; ... f(v1, ..., vn)
→
def f(x1, ..., xn) = B; ... [v1/x1, ..., vn/xn] B
```

In the above example, the call for `f` with actual parameter `f(v1, ..., vn)` can be evaluated by substituting all the occurrences of formal parameters `x1, x2, ..., xn` with `v1, v2, ..., vn`.

Let's see it in action in an example.

```scala
def iterate(n: Int, f: Int => Int, x: Int) =
if (n == 0) x else iterate(n-1, f, f(x))
def square(x: Int) = x * x
iterate(1, square, 3)
```

```scala
→ if (1 == 0) 3 else iterate(1-1, square, square(3))
→ iterate(0, square, square(3))
→ iterate(0, square, 3 * 3)
→ iterate(0, square, 9)
→ if (0 == 0) 9 else iterate(0-1, square, square(9)) → 9
```

Rewriting can be done anywhere in the term and all rewriting which terminate leads to same result. This is a important result of $\lambda-$ calculus, theory behind functional programming.

```scala
if (1 == 0) 3 else iterate(1 - 1, square, square(3))
iterate(0, square, square(3)) if (1 == 0) 3 // first way
else iterate(1 - 1, square, 3 * 3) //another way to rewrite
```

Both paths would give the same result. This property is called confluence (Church-Russev Theorem).

Let's introduce state in the objects. States can change over the course of time. An object has a state, if it's behavior is influenced by the history.

Every form of state is constructed from variable. In Scala, we use the keyword `var` to create variable. In practice object with state are represented as object with variables as members.

Here is an example of bank account:

```scala
class BankAccount {
private var balance = 0
def deposit(amount: Int): Unit = {
if (amount > 0) balance = balance + amount
}
def withdraw(amount: Int): Int =
if (0 < amount && amount <= balance) {
balance = balance - amount
balance
} else throw new Error("insufficient funds")
}
```

In [1]:
class BankAccount {
private var balance = 0
def deposit(amount: Int): Unit = {
if (amount > 0) balance = balance + amount
}
def withdraw(amount: Int): Int =
if (0 < amount && amount <= balance) {
balance = balance - amount
balance
} else throw new Error("insufficient funds")
}

defined class BankAccount

In [2]:
val acct = new BankAccount
acct deposit 50

acct: BankAccount = ammonite.$sess.cmd0$Helper$BankAccount@aed5c5

In [3]:
acct withdraw 20

res2: Int = 30

In [4]:
acct withdraw 40

: 

Clearly accounts are stateful objects. Effect of withdraw depends on the history of the object. Statefullness and variables are connected, let's how tight (or lossely) coupled.

Remember the definition of streams (lazy sequences). Instead of using a `lazy val`, we could also implement non-empty streams using a mutable variable:

```scala
def cons[T](hd: T, tl: => Stream[T]) = new Stream[T] {
def head = hd
private var tlOpt: Option[Stream[T]] = None
def tail: T = tlOpt match {
case Some(x) => x
case None => tlOpt = Some(tl); tail
}}
```
Is the result of  `cons` a stateful object?


It depends on the assumption you make on the rest of the system. One common assumption is streams are defined over purely functional computations. So the tail operation should not have side effects. In that case, optimization to cache `tail` and `tailOpt` to reuse it on all previous call to tails is purely a optimization that doesn't have a observable side effect outside the class of streams.

On the other hand, let's say tail could have a printing statement, then you would see that the second time tail is called in this string. It would come straight out of the cache, so there would be no side effect performed. Whereas, the first time, it would be called the operation would be performed, including the printing statement. So that means clearly the operation tail depends on the previous history of the object. It would be different depending on whether a previous tail was performed or not. So in that sense, the answer would be cons is a stateful object, provided that you also allow imperative side effect in computations for tail.

Consider the following class:

```scala
class BankAccountProxy(ba: BankAccount) {
def deposit(amount: Int): Unit = ba.deposit(amount)
def withdraw(amount: Int): Int = ba.withdraw(amount)
}
```

Question: Are instances of `BankAccountProxy` stateful objects?

Clearly, `BankAccountProxy` are stateful objects, because calling withdraw twice might lead to out of balance.